In [1]:
from pybis import Openbis
import json
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output

In [ ]:
# Global variables
OPENBIS_SESSION = None

In [ ]:
# Widgets
materials_dropdown = widgets.Dropdown(description='Select crystal:', disabled=False,)
materials_dropdown.style = {'description_width': 'initial'}
material_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '50%', height = '200px'))
material_metadata_boxes = widgets.HBox([materials_dropdown, material_details_textbox])
material_selection_radio_button = widgets.RadioButtons(description = 'Material', options=['No material', 'Crystal', 'Wafer substrate', '2D-layer material'], disabled = False)

sample_name_textbox = widgets.Text(value = '', placeholder = 'Enter sample name', description = 'Sample name:', disabled = False)
sample_name_textbox.style = {'description_width': 'initial'}
create_sample_button = widgets.Button(description = 'Save', disabled = False, button_style = 'success', tooltip = 'Save sample')
quit_button = widgets.Button(description = 'Main menu', disabled = False, button_style = 'danger', tooltip = 'Main menu')
bottom_buttons_hbox = widgets.HBox([create_sample_button, quit_button])

In [ ]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript('window.location.replace("home.ipynb")'))

# Function to create sample object inside openBIS using information selected in the app
def create_sample_action(b):
    global OPENBIS_SESSION
    
    if materials_dropdown.value == "-1" or material_selection_radio_button.value == "No material":
        sample_parents = []
    else:
        sample_parents = [materials_dropdown.value]
    
    OPENBIS_SESSION.new_object(type = "SAMPLE", 
                            collection = "20240716082516377-60",
                            props = {"$name": sample_name_textbox.value},
                            parents = sample_parents).save()

# Function to handle changes in the materials dropdown
def load_material_metadata(change):
    global OPENBIS_SESSION
    if materials_dropdown.value == "-1":
        material_details_textbox.value = ''
    else:
        if material_selection_radio_button.value == "Crystal":
            property_list = [("Name", "$name"), ("Material", "material"), ("Face", "face"), ("Sample Plate", "sample_plate"), ("Diameter", "diameter"), ("Height", "height"), ("Specifications", "specifications")]
        elif material_selection_radio_button.value == "Wafer substrate":
            property_list = [("Name", "$name"), ("Substrates", "substrates"), ("Sample Plate", "sample_plate"), ("Diameter", "diameter"), ("Height", "height"), ("Thickness", "thickness")]
        elif material_selection_radio_button.value == "2D-layer material":
            property_list = [("Name", "$name"), ("Sample Plate", "sample_plate"), ("2D Layers", "layers_2d"), ("Substrates", "substrates"), ("Width", "width"), ("Length", "length"), ("Thickness", "thickness")]
        
        material_metadata = OPENBIS_SESSION.get_object(materials_dropdown.value).props.all()
        material_metadata_string = ""
        for property in property_list:
            if property[1] in ["diameter", "height", "width", "thickness"]:
                if material_metadata[property[1]] is None:
                    material_metadata_string = f"{material_metadata_string} {property[0]}: {material_metadata[property[1]]}\n"
                else:
                    property_dict = json.loads(material_metadata[property[1]])
                    material_metadata_string = f"{material_metadata_string} {property[0]}: {property_dict['value']} {property_dict['unit']}\n"
            else:
                material_metadata_string = f"{material_metadata_string} {property[0]}: {material_metadata[property[1]]}\n"
        material_details_textbox.value = material_metadata_string

# Function to handle changes in the radio button
def select_material_radio_change(change):
    material_details_textbox.value = ''
    
    if material_selection_radio_button.value == "No material":
        clear_output()
        display(material_selection_radio_button)
        display(sample_name_textbox, bottom_buttons_hbox)
        
    else: 
        if material_selection_radio_button.value == "Crystal":
            materials = OPENBIS_SESSION.get_objects(type = "CRYSTAL")
            materials_dropdown.description = "Select crystal:"

        elif material_selection_radio_button.value == "Wafer substrate":
            materials = OPENBIS_SESSION.get_objects(type = "WAFER_SUBSTRATE")
            materials_dropdown.description = "Select wafer substrate:"
            
        elif material_selection_radio_button.value == "2D-layer material":
            materials = OPENBIS_SESSION.get_objects(type = "2D_LAYERED_MATERIAL")
            materials_dropdown.description = "Select 2D-layer material:"
            
        materials_names_permids = [(material.props["$name"], material.permId) for material in materials]
        materials_names_permids.insert(0, ('', "-1"))
        materials_dropdown.options = materials_names_permids
        materials_dropdown.observe(load_material_metadata, names = 'value')
            
        clear_output()
        display(material_selection_radio_button)
        display(material_metadata_boxes)
        display(sample_name_textbox, bottom_buttons_hbox)

In [ ]:
# Connect with openBIS
session_data = read_json("token.json")
OPENBIS_SESSION = Openbis(session_data["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(session_data["token"])

## Create a sample object in openBIS

In [ ]:
material_selection_radio_button.observe(select_material_radio_change, names='value')
create_sample_button.on_click(create_sample_action)
quit_button.on_click(close_notebook)
display(material_selection_radio_button)
display(sample_name_textbox, bottom_buttons_hbox)